In [2]:
import os
import torch

ModuleNotFoundError: No module named 'torch'

In [63]:
ZIPFILE = "./hymenoptera_data.zip"

In [64]:
from zipfile import ZipFile

In [65]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [58]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [66]:
data_dir = "./hymenoptera_data"

In [67]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [68]:
with ZipFile(ZIPFILE) as f:
    f.extractall()

In [70]:
#transforms
data_transform={
    "train":transforms.Compose([transforms.RandomResizedCrop(224),
                               transforms.RandomHorizontalFlip(),
                               transforms.ToTensor()
                               ]),
    "val":transforms.Compose([transforms.Resize(256),
                               transforms.CenterCrop(224),
                               transforms.ToTensor()
                               ])
}

In [71]:
image_dataset = {
    "train": datasets.ImageFolder(os.path.join(data_dir, "train"),transform=data_transform['train']), # mod
    "val": datasets.ImageFolder(os.path.join(data_dir, "val"),transform=data_transform['val']) # mod
}

In [72]:
dataLoader = {
    "train": torch.utils.data.DataLoader(image_dataset["train"], batch_size = 4, shuffle=True, num_workers=4),
    "val": torch.utils.data.DataLoader(image_dataset["val"], batch_size = 4, shuffle=True, num_workers=4)
}

In [73]:
data_sizes = {
    "train": len(image_dataset["train"]),
    "val": len(image_dataset["val"])
}
data_sizes

{'train': 244, 'val': 153}

In [74]:
class_names = image_dataset["train"].classes
class_names

['ants', 'bees']

In [75]:
model_conv = torchvision.models.resnet18(pretrained=True)

print(model_conv)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [76]:
# freezing the conv layers
for param in model_conv.parameters():
  param.required_grad = False

In [77]:
num_ftrs = model_conv.fc.in_features

num_ftrs

512

In [46]:
no_of_classes=len(class_names)
no_of_classes

2

In [79]:
no_of_classes = len(class_names)

model_conv.fc = nn.Linear(num_ftrs, no_of_classes) # incomming, outgoing edges 


In [80]:
print(model_conv)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [49]:
print(model_conv)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [81]:
model_conv = model_conv.to(device)

In [82]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_conv.fc.parameters(), lr=0.001)

In [83]:
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [84]:
def train_loop(model, loss_fn, optimizer, scheduler, num_epochs=20):

  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0

  for epoch in range(num_epochs):
    print(f"Epoch {epoch}/{num_epochs - 1}")
    print("=="*10)

    for phase in ["train", "val"]:
      training = phase == "train"
      if training:
        model.train() # sets the training mode
      else:
        model.eval() # sets the evaluation mode

      running_loss = 0.0
      running_corrects = 0

      for inputs, labels in dataLoader[phase]:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        with torch.set_grad_enabled(training):
          outputs = model(inputs)

          _, preds = torch.max(outputs, 1)
          loss = loss_fn(outputs, preds)

          if training:
            loss.backward()
            optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

      if training:
        scheduler.step()

      epoch_loss = running_loss / data_sizes[phase]
      epoch_acc = running_corrects.double() / data_sizes[phase] # mod

      print(f"{phase}, Loss: {epoch_loss}, accuracy: {epoch_acc}")
      if not training and epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_wts = copy.deepcopy(model.state_dict())
  
  model.load_state_dict(best_model_wts)
  return model

In [85]:
model = train_loop(model_conv, loss_fn, optimizer, exp_lr_scheduler, num_epochs=20)

Epoch 0/19
train, Loss: 0.018930667051388243, accuracy: 0.4918032786885246
val, Loss: 6.025515725852136e-05, accuracy: 0.5424836601307189
Epoch 1/19
train, Loss: 5.866893548418226e-05, accuracy: 0.4959016393442623
val, Loss: 5.2443241347764994e-05, accuracy: 0.5424836601307189
Epoch 2/19
train, Loss: 5.661090379690591e-05, accuracy: 0.4959016393442623
val, Loss: 6.35975114229855e-05, accuracy: 0.5424836601307189
Epoch 3/19
train, Loss: 8.76161183307394e-05, accuracy: 0.4959016393442623
val, Loss: 4.3176597179862094e-05, accuracy: 0.5424836601307189
Epoch 4/19
train, Loss: 4.663603013781182e-05, accuracy: 0.4959016393442623
val, Loss: 4.514756799276821e-05, accuracy: 0.5424836601307189
Epoch 5/19
train, Loss: 4.370349212921774e-05, accuracy: 0.4959016393442623
val, Loss: 3.147907544225032e-05, accuracy: 0.5424836601307189
Epoch 6/19
train, Loss: 5.57075557137933e-05, accuracy: 0.4959016393442623
val, Loss: 3.831238943937077e-05, accuracy: 0.5424836601307189
Epoch 7/19
train, Loss: 5.920